# The magic behind asyncio

What actually needs to be done in order to run several tasks in parallel without using threads is to make them interceptable, so that Python can execute parts of every task one after another and with that emulate paralle execution. This strategy is fine for many tasks.

- Tasks waiting for some input to arrive
- high level handling of I/O

It wont work for CPU intensive tasks. Here real parallelism is the onlu answer.

So how does it work? 

Basically we need to tell the iinterpreter, that a function execution needs to be treated differently, that there is places in the code, where execution can be intercepted. That's where the `async` keyword is for.

All actions that can be intercepted are marked with the `await` keyword. Behind it is a special implementation that allows interception.
    
    `time.sleep` vs `asyncio.sleep`

Thr result of both is exactly the same. Ones nature is blocking though, whilst the other allows interception to execute something else.

And finally you need a `runner` to execute the `async awaitables`. That's where `asyncio.run` comes into play.

Let's have a look at a very simple example with a blocking implementation that can be broken up to allow concurrency.

The idea is to transform a blocking `time.sleep` being just a simple placeholder for other more complex work with a blocking nature, into something that still executes the same blocking code, but allows things to happen in parallel.

In [ ]:
import asyncio
import time

Here's the blocking code to work on. Just a `time.sleep` statement to wait for `wait_for` seconds.

In [ ]:
async def blocking_wait_for(wait_for):
    print(f"I'm going to sleep right now for {wait_for} seconds.")
    tic = time.perf_counter()
    time.sleep(wait_for)
    toc = time.perf_counter()
    print(f"Hey y'all. That was a good nap! Slept for {toc-tic:.2f} seconds. Dig i miss something?")

And here's a simple notifier. We would love to see some message from it every now and then.

In [ ]:
async def notify_every(every):
    while keep_running:
        print("Notify!")
        await asyncio.sleep(every)

The `exit_after` function allows us to end execution of all the work after a chosen time.

In [ ]:
async def exit_after(exit_after):
    global keep_running
    await asyncio.sleep(exit_after)
    keep_running = False

And heres the final bit. We wait for the resut of every function using the `asyncio.gather` functiom. Let's see how it behaves

In [ ]:
keep_running = True
asyncio.gather(
    exit_after(5),
    notify_every(.2),
    blocking_wait_for(4),
)

That was a forseable outcome. `time.sleep` blocked the exeution globally, hence everything else was executed only after the `blocking_wait_for` finished.

But how can this be transformed into something more friendly?

All that needs to be done is to split our *long* `time.sleep` call into a series of many *shorter* `time.sleep` calls, which in the end add up to the same amount of sleep time. 

A call to `asyncio.sleep` with in minimalist sleep time allows intercepting the 'work' and execute other things in parallel.

The `friendly_blocking_wait_for` function immplements just this.

In [ ]:
async def friendly_blocking_wait_for(wait_for):
    print(f"I'm going to sleep right now for {wait_for} seconds.")
    # split blocking wait_for into parts
    number_of_parts=10
    sub_wait_for = wait_for / number_of_parts
    tic = time.perf_counter()
    for index in range(number_of_parts):
        # That call is still blocking!
        time.sleep(sub_wait_for)
        # This little 'pause' allows asyncio to trigger things in between
        await asyncio.sleep(1e-3)
    toc = time.perf_counter()
    print(f"Hey y'all. That was a good nap! Slept for {toc-tic:.2f} seconds. Dig i miss something?")

Let's check if that already helped.

In [ ]:
keep_running = True
asyncio.gather(
    exit_after(5),
    notify_every(.2),
    friendly_blocking_wait_for(2),
)

Much better!

So for every function that has to be interceptable a special implementatio allowing this needs to be available. 

And that's exactly the case. Find *doubles* of almost all blocking Python functions below the [asyncio part of Pythons documentation](https://docs.python.org/3/library/asyncio.html).